###  Makine Öğrenmesi ve Derin Öğrenme Kullanarak Ev Fiyatlarını Tahmin Etme

Modelimizi oluşturmak ve eğitmek için gerekli tüm kütüphaneleri import ediyoruz.


In [1]:
from keras.models import Sequential 
from keras.layers import Dense #kaç katman olduğunu, katmanlardaki nöron sayılarını ve aktivasyon fonksiyonlarını söyler
from sklearn.model_selection import train_test_split #test ve öğrenim için veriyi bölme
from sklearn.preprocessing import MinMaxScaler #to scale the data
import matplotlib.pyplot as plt
import pandas as pd


İlk olarak verilerini hazırlayıp yeni bir excel dosyasına yazdığımız veri setini okuyoruz. Ve info fonksiyonu ile veri setimiz hakkında gerekli olan bilgileri alıyoruz. 

In [4]:
df = pd.read_excel('/content/Veri_Kumesi_Edited.xlsx')
df.info()

df = df.drop("Unnamed: 0",axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     682 non-null    int64  
 1   Birim Fiyat    682 non-null    float64
 2   Oda Sayısı     682 non-null    int64  
 3   M2             682 non-null    int64  
 4   Bulunduğu Kat  682 non-null    int64  
 5   Bina Yaşı      682 non-null    int64  
 6   Kat Sayısı     682 non-null    int64  
 7   Isınma Tipi    682 non-null    int64  
 8   Konut Tipi     682 non-null    int64  
 9   Yakıt Şekli    682 non-null    int64  
 10  Tapu Tipi      682 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 58.7 KB


In [5]:
df

,Birim Fiyat,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Kat Sayısı,Isınma Tipi,Konut Tipi,Yakıt Şekli,Tapu Tipi
0,3166.6667,2,90,10,10,4,2,1,0,1
1,3000.0000,3,120,8,25,8,3,1,2,1
2,3000.0000,2,95,2,25,6,2,1,0,1
3,4172.7273,3,110,10,10,5,2,1,0,1
4,4450.0000,2,100,1,0,7,3,1,0,2
...,...,...,...,...,...,...,...,...,...,...
677,13.7500,1,80,10,5,6,2,1,0,1
678,10.6250,2,80,3,0,3,2,1,0,1
679,15.0000,1,40,1,4,5,4,1,0,1
680,11.5385,1,65,3,2,4,2,1,0,1


## Yapay Sinir Ağları Kullanarak Modelleme 

Bu bölümde verilerimizi yapay sinir ağlarını kullanarak modelleyeceğiz. Bunun için veri setini dataset adındaki listeye atıyoruz. 

In [9]:
dataset = df.to_numpy(int)
dataset

array([[3166,    2,   90, ...,    1,    0,    1],
       [3000,    3,  120, ...,    1,    2,    1],
       [3000,    2,   95, ...,    1,    0,    1],
       ...,
       [  15,    1,   40, ...,    1,    0,    1],
       [  11,    1,   65, ...,    1,    0,    1],
       [  10,    1,   60, ...,    1,    0,    1]])

Bu işlem sonrasında veri setimizi bağımlı ve bağımsız veri setlerine bölmemiz gerekiyor. Bu veri setindeki bağımlı değişkenimiz Birim Fiyat olacaktır. Yani bağımsız değişkenimizdeki veriler değiştikçe M2, Oda Sayısı, Bina Yaşı... bunlara uygun olarak birim fiyat değeri değişecektir. Bizim de bu modellemeden öğrenmek istediğimiz şey bu özellikler arasındaki bağlantıyı bulmak olacaktır. 

Bunun için X değişkenine 1. sütundan 9. sütuna kadar olan bağımsız değerleri veriyoruz. Ve Y için ise ilk sutun olan Birim Fiyattaki değerleri eklemiş olduk.

In [10]:
X = dataset[:, 1:9] 

Y = dataset[:, 1] 

Burdan sonra, tüm özelliklerin(features) 0 ve 1 değerleri arasında olması için preprocessing içerisindeki min_max_scaler fonksiyonunu kullanıyoruz.  

In [11]:
min_max_scaler = MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.1       , 0.08387097, 1.        , ..., 0.4       , 0.2       ,
        0.        ],
       [0.2       , 0.12258065, 0.8       , ..., 0.6       , 0.2       ,
        1.        ],
       [0.1       , 0.09032258, 0.2       , ..., 0.4       , 0.2       ,
        0.        ],
       ...,
       [0.        , 0.01935484, 0.1       , ..., 0.8       , 0.2       ,
        0.        ],
       [0.        , 0.0516129 , 0.3       , ..., 0.4       , 0.2       ,
        0.        ],
       [0.        , 0.04516129, 0.3       , ..., 0.6       , 0.2       ,
        0.        ]])

Bir sonraki adımda verimizi %80 eğitim ve %20(%10 eğitim %10 değerlendirme) test ve değerlendirmede kullanacağız. Bunun için gerekli değişkenleri de oluşturuyoruz.

In [14]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale , Y, test_size=0.2)

X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
# Train setimiz içerisinde 545 veri noktası, test ve değerlendirme setimizde ise 68 adet veri noktası var.
# Ve X değerlerinin 8 adet özelliği varken Y değerlerin sadece tahmin edilecek bir özelliği var.

print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(545, 8) (68, 8) (69, 8) (545,) (68,) (69,)


Şimdi ise bir sonraki adımımız olan modeli ve derin sinir ağını modellemeye geçiyoruz.

In [23]:
model = Sequential() # Yapay sinir ağını başlatıyoruz.

model.add(Dense(units=32, activation='sigmoid', input_dim=8))
model.add(Dense(units=32, activation='sigmoid'))
model.add(Dense(units=32, activation='sigmoid'))
model.add(Dense(units=1, activation='relu'))

In [34]:
# Kayıp fonksiyonu bize ne kadar iyi bir eğitim alındığını gösteriyor ve optimizer fonksiyonunu kullanarak değerleri optimize ediyor.

model.compile(optimizer='sgd',
              loss = "binary_crossentropy",
              metrics = ["accuracy"]
              )

In [35]:
# Modeli Eğitmeye başlıyoruz.

hist = model.fit(
           X_train, Y_train,
           batch_size = 32, epochs = 100,
           validation_data = (X_val, Y_val)
)

Epoch 1/100
18/18 [==============================] - 1s 13ms/step - loss: 32.4335 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 2/100
18/18 [==============================] - 0s 4ms/step - loss: 32.6821 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 3/100
18/18 [==============================] - 0s 4ms/step - loss: 33.8407 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 4/100
18/18 [==============================] - 0s 4ms/step - loss: 33.9341 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 5/100
18/18 [==============================] - 0s 4ms/step - loss: 33.2392 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 6/100
18/18 [==============================] - 0s 4ms/step - loss: 32.8687 - accuracy: 0.0000e+00 - val_loss: 32.4378 - val_accuracy: 0.0000e+00
Epoch 7/100
18/18 [==============================] - 0s 4ms/step - loss: 32.9307 - accuracy: 

In [26]:
model.evaluate(X_test, Y_test)[1]

3/3 [==============================] - 0s 4ms/step - loss: 33.0854 - accuracy: 0.0000e+00


0.0

In [29]:
# Bu konuda bir tahmin yapmaya çalışıyoruz.

prediction = model.predict(X_test)
prediction = [1 if y>=.10 else 0 for y in prediction]
print(prediction)
print(Y_test)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3 2 3 3 2 1 3 1 3 1 1 3 3 1 2 3 3 4 1 1 2 2 3 1 3 2 3 3 3 2 2 1 2 2 2 3 2
 1 1 1 5 1 1 1 4 2 2 1 3 3 1 1 1 3 2 2 3 3 2 2 1 3 1 2 4 2 1 3 3]


## En İyi Algoritmayı Bulma

Yapay sinir ağının modelinin bu veri setini tahmin etmede pek de iyi olmadığını görüyoruz. Bu nedenle en iyi tahmini verecek algortimayı bulmak için aşağıdaki şekilde tekrardan modellerimizi eğiterek algoritma analizi yapıyoruz.

In [36]:
df.shape

(682, 10)

In [37]:
X = df.drop(['Birim Fiyat'],axis='columns')
X.head(3)

,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Kat Sayısı,Isınma Tipi,Konut Tipi,Yakıt Şekli,Tapu Tipi
0,2,90,10,10,4,2,1,0,1
1,3,120,8,25,8,3,1,2,1
2,2,95,2,25,6,2,1,0,1


In [38]:
X.shape

(682, 9)

In [40]:
y = df["Birim Fiyat"]
y.head(5)

0    3166.6667
1    3000.0000
2    3000.0000
3    4172.7273
4    4450.0000
Name: Birim Fiyat, dtype: float64

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [42]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.19759155873217815

In [43]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.14099252, 0.24362815, 0.28621775, 0.25220046, 0.10049839])

In [44]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)


,model,best_score,best_params
0,linear_regression,0.204707,{'normalize': True}
1,lasso,0.205956,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,-0.272632,"{'criterion': 'friedman_mse', 'splitter': 'ran..."


Görüldüğü üzere bu veri setindeki en iyi tahmini Linear regression yöntemi ile almışız.